In [206]:
import pandas as pd
import numpy as np

In [358]:
df = pd.read_csv("/Users/timholdsworth/code/scaling-science/Data/impact_1900-1950.csv")

In [374]:
df = df.head(5)

In [360]:
# Takes in a series of values and weighs them
def time_weighted_score(df):
    # Add up all the values in a row, which are all the paper's pagerank scores in given years
    df['score_sum'] = df.sum(axis=1, skipna=True)
    # Define time_since_pub as the number of years in the graph - the number of values that are not NaN
    df['time_since_pub'] = len(df.columns) - 4 - df.isnull().sum(axis=1, skipna=True) 
    df['weighted_score'] = df['score_sum'] / df['time_since_pub']
    df = df.rename(columns={"a.title": "title"})
    return df

In [370]:
df = time_weighted_score(df).sort_values(by=['weighted_score'], ascending=False)

In [372]:
# Get the number of years since publication as a series for all rows in database
time_since_pub = df.loc[:, 'time_since_pub']
# Becauset there are 3 extra column at the end
start_col_pos = time_since_pub + 3
# Because we want the last certain number of columns
start_col = -start_col_pos

In [364]:
# TODO figure out why multiplying these two gives a series double the length of each of the series being multiplied
def exp_decay(df, start_col):
    for index, row in df.iterrows():
    
        # Get the column which the data starts for this row 
        start = start_col[index]
        # Create a series of the impact scores for this row
        impact_scores = df.iloc[index, start:-3].reset_index(drop=True)
       
        print('Printing Impact score')
        print(impact_scores)
        print('Impact score is length ' + str(len(impact_scores)))
        
        # Create an Series with decay coefficients that is the same length as the impact_scores series
        time = np.arange(len(impact_scores))
        decay_list = [np.exp(-t / 25) for t in time]
        decay_series = pd.Series(decay_list)
        
        print('Printing decay series')
        print(decay_series)
        print('Decay series is length ' + str(len(decay_series)))
        
        decayed_score = decay_series.multiply(impact_scores)
        
        print('Printing decayed score')
        # TODO figure out why there are NaN's printing out
        print(decayed_score)
        print('Decayed score is length ' + str(len(decayed_score)))
        
        # Write the results back to the appropriate column in dataframe
        # TODO figure out how to do this without doing it inplace
        #df.iloc[index, start:-3] = decayed_score

    return df
    
df1 = exp_decay(df, start_col)

Printing Impact score
0     0.15
1     0.15
2     0.15
3     0.15
4     0.15
5     0.15
6     0.15
7     0.15
8     0.15
9     0.15
10    0.15
11    0.15
12    0.15
13    0.15
14    0.15
15    0.15
Name: 3, dtype: object
Impact score is length 16
Printing decay series
0     1.000000
1     0.960789
2     0.923116
3     0.886920
4     0.852144
5     0.818731
6     0.786628
7     0.755784
8     0.726149
9     0.697676
10    0.670320
11    0.644036
12    0.618783
13    0.594521
14    0.571209
15    0.548812
dtype: float64
Decay series is length 16
Printing decayed score
0          0.15
1      0.144118
2      0.138467
3      0.133038
4      0.127822
5       0.12281
6      0.117994
7      0.113368
8      0.108922
9      0.104651
10     0.100548
11    0.0966055
12    0.0928175
13    0.0891781
14    0.0856814
15    0.0823217
dtype: object
Decayed score is length 16
Printing Impact score
0      NaN
1      NaN
2      NaN
3     0.15
4     0.15
5     0.15
6     0.15
7     0.15
8     0.15
9     0.1

In [324]:
#df1.to_csv('/Users/timholdsworth/code/scaling-science/Data/1000_most_impactful_papers_decayed.csv')

In [373]:
# Extra code I used to test out multiplying a series
# An array of values from 0 to (100 - the number of years a paper has been published)
s = pd.Series([10, 10, 10, 10])
time = np.arange(len(s))
# Array of coefficients that get smaller as time goes on
decay_coeff = [np.exp(-t / 15) for t in time] 
#print(decay_coeff)

# Multiple a given series by its corresponding decay coefficients
s.multiply(decay_coeff)

0    10.000000
1     9.355070
2     8.751733
3     8.187308
dtype: float64